In [93]:
import pandas as pd
import numpy as np
train = pd.read_csv("train_df.csv")
test = pd.read_csv("test_df.csv")

In [94]:
train.head()

,search_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,target
0,758,9,0,0,1,20,3,40,0,3,...,0.204682,0.271755,0.055623,0,0,0,0.38648,0.0,0.0,0
1,758,9,0,0,1,20,3,40,0,3,...,0.195531,0.188787,0.036914,0,0,0,0.10982,0.0,0.0,0
2,758,9,0,0,1,20,3,40,0,3,...,0.148609,0.186517,0.027718,0,0,0,0.03674,0.0,0.0,0
3,758,9,0,0,1,20,3,40,0,3,...,0.223748,0.229039,0.051247,0,0,0,0.00000,0.0,0.0,0
4,758,9,0,0,1,20,3,40,0,3,...,0.170935,0.249031,0.042568,0,0,0,0.00000,0.0,0.0,0


In [95]:
test.head()

,search_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,target
0,10655,9,0,0,1,20,4,40,0,0,...,0.148830,0.196644,0.029267,0,0,0,0.03674,0.0,0.0,0
1,10655,9,0,0,1,20,4,40,0,0,...,0.119724,0.174199,0.020856,0,0,0,0.00000,0.0,0.0,0
2,10655,9,0,0,1,20,4,40,0,0,...,0.160606,0.198780,0.031925,0,0,0,0.00000,0.0,0.0,0
3,10655,9,0,0,1,20,4,40,0,0,...,0.180191,0.187882,0.033855,0,0,0,0.00000,0.0,0.0,0
4,10655,9,0,0,1,20,4,40,0,0,...,0.117308,0.153586,0.018017,0,0,0,0.00000,0.0,0.0,0


In [96]:
train['target'].value_counts(normalize=True)

0    0.978649
1    0.021351
Name: target, dtype: float64

In [97]:
test['target'].value_counts(normalize=True)

0    0.977763
1    0.022237
Name: target, dtype: float64

Очевидно, классы несбалансированы, и из-за разреженных данных древовидные модели использовать не имеет большого смысла

In [158]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train = train.drop(columns=['search_id', 'target'])
y_train = train['target']

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = test.drop(columns=['search_id', 'target'])
y_test = test['target']

X_test = scaler.transform(X_test)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

model = linear_model.Lasso(alpha = 0.1)

Будем использовать Lasso + искусственно увеличим размер обучающей выборки, чтобы сбалансировать классы

In [159]:
from imblearn.over_sampling import ADASYN

adasyn = ADASYN()
X_train_r, y_train_r = adasyn.fit_resample(X_train, y_train)
X_val_r, y_val_r = adasyn.fit_resample(X_val, y_val)

In [160]:
y_train_r.value_counts(normalize=True)

1    0.501735
0    0.498265
Name: target, dtype: float64

In [161]:
model.fit(X_train_r, y_train_r)

predictions = model.predict(X_val_r)

In [162]:
from sklearn.metrics import ndcg_score
from sklearn.metrics import roc_auc_score


def calculate_ndcg(y_true, y_score):
    true_relevance = np.asarray([y_true])
    scores = np.asarray([y_score])
    ndcg = ndcg_score(true_relevance, scores)
    return ndcg


calculate_ndcg(y_val_r, predictions)


0.9293679637351299

In [163]:
calculate_ndcg(y_test, model.predict(X_test))

0.6292937676421839

In [164]:
roc_auc_score(y_test, model.predict(X_test))

0.7787822152272279

Посмотрим, какие признаки отобрала Lasso

In [166]:
count = 0
for i in model.coef_:
    if i != 0:
        count += 1

print(f'Модель отобрала {count} признаков')

print(model.coef_)

Модель отобрала 1 признаков
[ 0.         -0.         -0.         -0.         -0.          0.
  0.         -0.         -0.          0.          0.          0.
  0.         -0.         -0.         -0.         -0.         -0.
  0.          0.         -0.          0.         -0.         -0.
 -0.          0.         -0.          0.          0.          0.
  0.          0.          0.          0.         -0.          0.
 -0.         -0.          0.          0.          0.          0.
  0.          0.         -0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -0.         -0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.         -0.          0.06667727  0.
  0.          0.          0.          0.          0.          0.
  0.        ]
